# Scientific American

Scrapes Scientific American site.

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import configparser
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

configs = configparser.ConfigParser()
configs.read('config.ini')

['config.ini']

Define scraping functions:

In [6]:
def initialize(download_dir):
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': download_dir,
             'plugins.always_open_pdf_externally': True}
    options.add_experimental_option('prefs', prefs)
    return webdriver.Chrome(options=options, executable_path='../chromedriver')

In [22]:
# Initialize browser
download_path = '/Volumes/ARCHIVES/GitHub/scientific-american/issues'
browser = initialize(download_path)
wait = WebDriverWait(browser, 15)

# Navigate to page with search results
current_url = 'https://www.scientificamerican.com/store/archive/?magazineFilterID=Scientific+American+Magazine'
browser.get(current_url)

# Log into my account 
signin_button_present = EC.presence_of_element_located((By.ID, 'signin-click'))
wait.until(signin_button_present)
signin_button = browser.find_element(By.ID, 'signin-click')
signin_button.click()
email_input = browser.find_element(By.ID, 'emailAddress1')
password_input = browser.find_element(By.ID, 'password1')
browser.execute_script('arguments[0].value = "' + configs['SECRET']['EMAIL'] + '";', email_input) 
browser.execute_script('arguments[0].value = "' + configs['SECRET']['PASSWORD'] + '";', password_input)
login_button = browser.find_element(By.XPATH, '/html/body/div[4]/div/header/nav[1]/div/div[3]/ul/li[2]/section/form/button')
time.sleep(2)
login_button.click()
signed_in = EC.invisibility_of_element_located((By.XPATH, '/html/body/div[4]/div/header/nav[1]/div/div[3]/ul/li[2]/section/form/button'))
wait.until(signed_in)

# Collect all download links on the current page
download_links = []
download_buttons = browser.find_elements_by_class_name('download')
for btn in download_buttons:
    download_links.append(btn.get_property('href'))

browser.get(download_links[0])